<a href="https://colab.research.google.com/github/scope-lab-vu/transactive-blockchain/blob/master/TCPS_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These commands set up git large file storage. Not sure if we need it, so they are commented out for now.

In [0]:
# ! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

In [0]:
# ! wget https://github.com/git-lfs/git-lfs/releases/download/v2.10.0/git-lfs-linux-amd64-v2.10.0.tar.gz

In [0]:
# ! tar -xvf git-lfs-linux-amd64-v2.10.0.tar.gz ; chmod +X install.sh ; ./install.sh 

Mount Google Drive to access installation files

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Install cplex from Google Drive. installer.properties included in directory for silent install

In [2]:
! chmod u+x /content/gdrive/My\ Drive/ColabLibraries/cplex_studio129.linux-x86-64.bin

!/content/gdrive/My\ Drive/ColabLibraries/cplex_studio129.linux-x86-64.bin -f "installer.properties"


Preparing to install
Extracting the JRE from the installer archive...
Unpacking the JRE...
Extracting the installation resources from the installer archive...
Configuring the installer for this system's environment...

Launching installer...



Install cplex python binaries

In [0]:
%%capture
! cd /opt/ibm/ILOG/CPLEX_Studio129/python/; python3 setup.py install;

Clone transax repo

In [5]:
from getpass import getpass
import os

user = getpass('GitHub User')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password

!git clone https://$GITHUB_AUTH@github.com/scope-lab-vu/transactive-blockchain.git


GitHub User··········
GitHub password··········
Cloning into 'transactive-blockchain'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 4762 (delta 10), reused 4 (delta 1), pack-reused 4740
Receiving objects: 100% (4762/4762), 139.49 MiB | 33.22 MiB/s, done.
Resolving deltas: 100% (2178/2178), done.
Checking out files: 100% (2097/2097), done.


Install transax library

In [6]:
%%capture
! cd /content/transactive-blockchain/transax; sudo pip3 install .

Processing /content/transactive-blockchain/transax
     |████████████████████████████████| 225kB 3.5MB/s 
  Created wheel for transax: filename=transax-1.0-cp36-none-any.whl size=9784 sha256=f49a5d5d6917f8e706c1199f6141f7b0bcc816bfbc614891a384ab1026273194
  Stored in directory: /tmp/pip-ephem-wheel-cache-l7eumhj2/wheels/ca/ea/6e/796640569df0e9318c2fd2949c5825679a6e1b6ac45b956177
  Created wheel for pycurl: filename=pycurl-7.43.0.5-cp36-cp36m-linux_x86_64.whl size=283550 sha256=1bad6fd651da2b2543f67ba85e9012276d0ce3c509d71dd73dc82cb014804a27
  Stored in directory: /root/.cache/pip/wheels/98/a3/19/9439c79b88a742f508af0c4e410ae3f685db2e13d3374e798e
Successfully built transax pycurl


In [7]:
import cplex
from transax.Microgrid import Microgrid
from transax.MatchingSolver import Offer, MatchingSolver
microgrid = Microgrid(interval_length=1.0, C_ext=20.0, C_int=25.0, feeders=[0, 1], prosumer_feeder={
    0: 0,
    1: 0,
    2: 0,
    3: 1,
    4: 1,
    5: 1
  })
solver = MatchingSolver(microgrid)
buying_offers = [
  Offer(0, 0, 1, 10, 5, 1),
  Offer(1, 1, 6, 15, 5, 1),
  Offer(2, 3, 1, 10, 5, 1),
  Offer(3, 4, 6, 15, 5, 1),
]
selling_offers = [
  Offer(4, 2, 1, 15, 15, 1),
  Offer(5, 5, 1, 15, 5, 1),
]
(trades, objective) = solver.solve(buying_offers=buying_offers, selling_offers=selling_offers)
print("Success: {} energy traded".format(objective))

Success: 20.0 energy traded
